# Goal : Scan the 3D predcitions of the FP with the Dpo fold database. 

### A. Run Foldseek
### B. Get the hits with good proba
### C. Report the folds
***

# A

In [1]:
import subprocess
import os

path_project = "/media/concha-eloko/Linux/PhageDEPOdetection"
path_pdb = f"{path_project}/FP_deposcope_pdb_2303"
path_db = f"/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    dir_out = f"{path_project}/FP_deposcope_pdb_2303_Foldseek"
    protein_id = path_in.split("/")[-1].split(".pdb")[0]
    path_out = f"{dir_out}/{protein_id}.out"
    output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
    seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
    #seek = f"foldseek easy-search {path_in} {path_db} {path_out}.html {path_tmp} --format-mode 3"
    seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    press_out, press_err = seek_process.communicate()
    print (press_out, press_err)

paths = [f"{path_pdb}/{file}" for file in os.listdir(f"{path_pdb}")]

for path in paths :
    seek_beta_helix(path)





b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\n' None
b'/bin/sh: 1: foldseek: not found\

***
## B  

> Get a list of the failed Dpos

In [8]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/PhageDEPOdetection"
dir_out = f"{path_project}/FP_deposcope_pdb_2303_Foldseek"

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

FP_results = {}
no_structure = []
outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]

for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        fold = info_df[info_df["ID"] == row["target"]]["Folds"].values[0]
        if fold != "jelly-roll" :
            if row["prob"] >= 0.2 :
                FP_results[row["query"]]= (row["prob"],row["target"] , fold)
                break
    else :
        no_structure.append(row["query"])
                


In [9]:
import pprint
pp = pprint.PrettyPrinter(width = 150, sort_dicts = True, compact = False)
pp.pprint(FP_results)




{'CP000711.1__ABQ88395.pdb': (0.912, 'AF-A0A7C4BP91-F1-model_v4.pdb', '6-bladed beta-propeller'),
 'FQ482084.1__CBX44498.pdb': (0.632, 'AF-A0A654UB90-F1-model_v4.pdb', '6-bladed beta-propeller'),
 'GQ413937.1__ACY66694.pdb': (0.353, 'AF-A0A654UB90-F1-model_v4.pdb', '6-bladed beta-propeller'),
 'GQ413937.1__ACY66700.pdb': (1.0, 'K17alfa62__cds_64_A_3_129_548.pdb', 'right-handed beta-helix'),
 'HQ632825.1__AGN12247.pdb': (1.0, 'K39PH122C2__cds_55_A_5_286_658.pdb', 'right-handed beta-helix'),
 'HQ632825.1__AGN12298.pdb': (0.382, 'AF-A0A0R3Q9P4-F1-model_v4.pdb', 'right-handed beta-helix'),
 'HQ632825.1__AGN12307.pdb': (0.949, 'AF-A0A562WF65-F1-model_v4.pdb', 'right-handed beta-helix'),
 'KC821608.1__AGO47494.pdb': (1.0, 'K17alfa62__cds_64_A_3_129_548.pdb', 'right-handed beta-helix'),
 'KC821610.1__AGO47761.pdb': (0.411, 'AF-A0A7C4BP91-F1-model_v4.pdb', '6-bladed beta-propeller'),
 'KC821629.1__AGO49269.pdb': (0.411, 'AF-A0A7C4BP91-F1-model_v4.pdb', '6-bladed beta-propeller'),
 'KC821630.1_

In [10]:
len(no_structure) , len(FP_results)

(44, 78)

In [17]:
from collections import Counter

folds_FPs = [FP_results[prot][2] for prot in FP_results]
Counter(folds_FPs)

Counter({'right-handed beta-helix': 41, '6-bladed beta-propeller': 37})

In [1]:
import nglview as nv

def view_pdb(file) : 
    path_project = "/media/concha-eloko/Linux/PhageDEPOdetection"
    path_pdb = f"{path_project}/FP_2108"
    view = nv.show_structure_file(f"{path_pdb}/{file}")
    view


In [2]:
path_project = "/media/concha-eloko/Linux/PhageDEPOdetection"
path_pdb = f"{path_project}/FP_pdb"

view = nv.show_file(f"{path_pdb}/KC821625.1_prot_AGO49026.1_15.pdb")
view.background = 'black'
view

NGLWidget(background='black')

In [28]:
no_structure

['NC_017972.1_prot_YP_006382630.1_100.pdb',
 'NC_016071.1_prot_YP_004893846.1_39.pdb',
 'KC821625.1_prot_AGO49026.1_15.pdb',
 'NC_019401.1_prot_YP_006987325.1_223.pdb',
 'NC_027364.1_prot_YP_009150318.1_4.pdb',
 'NC_019526.1_prot_YP_007007689.1_534.pdb',
 'NC_019929.1_prot_YP_007237854.1_204.pdb',
 'NC_006883.2_prot_YP_214266.1_30.pdb',
 'NC_025422.1_prot_YP_009098968.1_210.pdb',
 'NC_027364.1_prot_YP_009150602.1_288.pdb',
 'KC821621.1_prot_AGO48656.1_21.pdb',
 'HQ632825.1_prot_AGN12459.1_301.pdb',
 'NC_027364.1_prot_YP_009150315.1_1.pdb',
 'NC_017971.2_prot_YP_006382482.1_22.pdb',
 'JX262376.1_prot_AFO10887.1_21.pdb',
 'AY349011.3_prot_AAQ55000.1_65.pdb',
 'NC_006883.2_prot_YP_214361.1_128.pdb',
 'NC_019526.1_prot_YP_007007233.1_78.pdb',
 'NC_023557.1_prot_YP_009004812.1_20.pdb',
 'NC_003157.5_prot_NP_438158.1_46.pdb',
 'NC_019917.1_prot_YP_007236806.1_60.pdb',
 'HQ632825.1_prot_AGN12302.1_144.pdb',
 'NC_020079.1_prot_YP_007348548.1_180.pdb',
 'NC_027364.1_prot_YP_009150606.1_292.pdb'